In [1]:
# !pip install torchsummary

In [2]:
##code
"""
Target : Build lighter model to check the lower limit
Result : Total params - 3,258
Analysis: training slow
"""

'\nTarget : increase accuracy at 15th epoch, keeping same parameters\nResult : Total params - 3,342, test: 98.89%%, train 98.61% at 15th rpoch\nAnalysis: training speed improved\n'

In [3]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np

In [4]:
dropout_value = 0.1
class Net(nn.Module):
    def __init__(self):
        super().__init__()

        self.convblock1 = nn.Sequential(
          nn.Conv2d(in_channels=1, out_channels=2 ,kernel_size=(3,3),padding=0, bias=False), ##>>40 param, 28 input/channel size
          nn.ReLU(),
          nn.Conv2d(in_channels=2, out_channels=4 ,kernel_size=(3,3),padding=0, bias=False), ##>>148 , 26
          nn.ReLU(),
          nn.Conv2d(in_channels=4, out_channels=8 ,kernel_size=(3,3),padding=0, bias=False), ##>>148, 24
          nn.ReLU(),
        )
        self.pool1 = nn.Sequential( ##>>0, 12
          nn.MaxPool2d(2,2),
        )
        self.convblock2 = nn.Sequential(
          nn.Conv2d(in_channels=8, out_channels=2 ,kernel_size=(3,3),padding=1, bias=False), ##>>298 , 10
          nn.ReLU(),
          nn.Conv2d(in_channels=2, out_channels=4 ,kernel_size=(3,3),padding=1, bias=False), ##>>614 , 10
          nn.ReLU(),
          nn.Conv2d(in_channels=4, out_channels=8 ,kernel_size=(3,3),padding=0, bias=False), ##>>614 , 8
          nn.ReLU(),        
        )
        self.convblock3 = nn.Sequential(
          nn.Conv2d(in_channels=8, out_channels=2 ,kernel_size=(3,3),padding=1, bias=False), ##>>298 , 8
          nn.ReLU(),
          nn.Conv2d(in_channels=2, out_channels=4 ,kernel_size=(3,3),padding=1, bias=False), ##>>614 , 8
          nn.ReLU(),
          nn.Conv2d(in_channels=4, out_channels=8 ,kernel_size=(3,3),padding=0, bias=False), ##>>614 , 6
          nn.ReLU(),
        )
        self.pool2 = nn.Sequential( ##>>0, 3
          nn.MaxPool2d(2,2),
        )
        self.convblock4 = nn.Sequential(
          nn.Conv2d(in_channels=8, out_channels=8 ,kernel_size=3,padding=1, bias=False), ##>>1168 , 3
          nn.ReLU(),
          nn.Conv2d(in_channels=8, out_channels=8 ,kernel_size=3,padding=1, bias=False), ##>>2320 , 3
          nn.ReLU(),
          nn.Conv2d(in_channels=8, out_channels=10 ,kernel_size=3,padding=0, bias=False), ##>>1450 , 1
          # nn.ReLU()
        )

    def forward(self, x):
        x = self.convblock1(x)
        # print (x.shape)
        x = self.pool1(x)
        # print (x.shape)
        x= self.convblock2(x)
        # print (x.shape)
        x= self.convblock3(x)
        # print (x.shape)
        x= self.pool2(x)
        # print (x.shape)
        x= self.convblock4(x)
        # print (x.shape)
        x = x.view(-1,10)
        x = F.log_softmax(x)
        return x 


In [5]:
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 2, 26, 26]              18
              ReLU-2            [-1, 2, 26, 26]               0
            Conv2d-3            [-1, 4, 24, 24]              72
              ReLU-4            [-1, 4, 24, 24]               0
            Conv2d-5            [-1, 8, 22, 22]             288
              ReLU-6            [-1, 8, 22, 22]               0
         MaxPool2d-7            [-1, 8, 11, 11]               0
            Conv2d-8            [-1, 2, 11, 11]             144
              ReLU-9            [-1, 2, 11, 11]               0
           Conv2d-10            [-1, 4, 11, 11]              72
             ReLU-11            [-1, 4, 11, 11]               0
           Conv2d-12              [-1, 8, 9, 9]             288
             ReLU-13              [-1, 8, 9, 9]               0
           Conv2d-14              [-1, 

/var/folders/m2/x6hw0bg17hd2xt6rzl4jd04h0000gn/T/ipykernel_58146/4062518061.py:59: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  x = F.log_softmax(x)


In [6]:
transforms_ = transforms.Compose([
                        transforms.RandomRotation((-7,7),fill=1),
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])

In [7]:
torch.manual_seed(1)
batch_size = 64

kwargs = {'num_workers': 7, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
    transform=transforms.Compose([
                        # transforms.RandomRotation((-7,7),fill=1),
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [8]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    correct=0
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.float().to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')
        pred = output.argmax(dim=1)
        correct+=pred.eq(target).sum().item()
    acc = 100*correct/len(train_loader.dataset)
    print(f'Train Accuracy: {acc:.2f}%')
        
def test(model, device, test_loader,epoch):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for _,(data, target) in enumerate(test_loader):
            data, target = data.float().to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)

    print('Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [9]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
for epoch in range(1, 15):
    print(f'Epoch: {epoch}')
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader,epoch)


Epoch: 1


  0%|          | 0/938 [00:00<?, ?it/s]/var/folders/m2/x6hw0bg17hd2xt6rzl4jd04h0000gn/T/ipykernel_58146/4062518061.py:59: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  x = F.log_softmax(x)
loss=2.302605152130127 batch_id=937: 100%|██████████| 938/938 [00:21<00:00, 44.22it/s] 


Train Accuracy: 6.47%
Test set: Average loss: 2.3026, Accuracy: 689/10000 (6.89%)
Epoch: 2


loss=2.3026022911071777 batch_id=937: 100%|██████████| 938/938 [00:21<00:00, 42.83it/s]


Train Accuracy: 7.12%
Test set: Average loss: 2.3026, Accuracy: 769/10000 (7.69%)
Epoch: 3


...

loss=2.3025894165039062 batch_id=937: 100%|██████████| 938/938 [00:20<00:00, 44.79it/s]


Train Accuracy: 8.60%
Test set: Average loss: 2.3026, Accuracy: 941/10000 (9.41%)
Epoch: 5


loss=2.3025739192962646 batch_id=937: 100%|██████████| 938/938 [00:21<00:00, 43.51it/s]


Train Accuracy: 9.60%
Test set: Average loss: 2.3026, Accuracy: 1040/10000 (10.40%)
Epoch: 6


loss=2.302572250366211 batch_id=864:  92%|█████████▏| 863/938 [00:20<00:01, 41.70it/s] 